In [1]:
import os
import types
import pandas as pd
import zipfile

import ibm_boto3
from ibm_botocore.client import Config

In [2]:
# The code was removed by DSX for sharing.

In [3]:
# create an HTTP client for reading and writing files to and from Cloud Ojbect Store
cosClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [4]:
# this works for reading an Excel file from COS ..............

# get byte stream for reading Excel file from Cloud Object Store
excelStream = cosClient.get_object(Bucket=credentials['BUCKET'], Key='report_IGA_Global_Q1_2016.xlsx')['Body']

# add an iterator method to the stream object so pandas will accept it as a file-like object
def __iter__(self): return 0
if not hasattr(excelStream, "__iter__"): excelStream.__iter__ = types.MethodType(__iter__, excelStream) 

# prepare to read the byte stream from Cloud Object Store as an Excel file
excelFile = pd.ExcelFile(excelStream)

# concatenate the data in each spreadsheet of the Excel file into a single pandas DataFrame
ibmData = pd.concat( map(lambda sheet: pd.read_excel(excelFile, sheet, header=0), excelFile.sheet_names) )

In [5]:
# this works for writing a string to COS ............
helloStream = cosClient.put_object(Bucket=credentials['BUCKET'], Key='hellotrial.txt', Body=b'hello, world')

In [6]:
# this works for writing a CSV file to COS ............
os.makedirs('merged', exist_ok=True)
ibmData.to_csv('merged/ibmData.csv', index=False, float_format='%.9g', columns=ibmData.columns)
cosClient.upload_file(Filename='merged/ibmData.csv', Bucket=credentials['BUCKET'], Key='ibmData.csv')

In [7]:
# this works for writing a ZIP file to COS ............
with zipfile.ZipFile('ziptrial.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipFile:
    for file in os.listdir('merged'):
        zipFile.write('merged/'+file, file)
cosClient.upload_file(Filename='ziptrial.zip', Bucket=credentials['BUCKET'], Key='ziptrial.zip')

In [35]:
# this fails ...........
with io.BytesIO() as helloBuffer:
    helloStream = cosClient.put_object(Bucket=credentials['BUCKET'], Key='hellotrial.txt', Body=helloBuffer)
    helloBuffer.write(b"hello, world\n")
    helloBuffer.write(b"goodbye, world\n")    

In [39]:
# this throws exception ...........
helloBuffer = io.BytesIO()
helloBuffer.write(b"hello, world\n")
helloBuffer.write(b"goodbye, world\n")
helloStream = cosClient.put_object(Bucket=credentials['BUCKET'], Key='hellotrial.txt', Body=helloBuffer)
helloBuffer.write(b"still there?\n")
helloBuffer.close()

KeyboardInterrupt: 

In [54]:
# this throws exception ...........
with io.BytesIO() as buffer:
     credentcosObject = cosClient.upload_fileobj(buffer,ials['BUCKET'], 'ziptrial.zip')
    with zipfile.ZipFile(buffer, 'w', compression=zipfile.ZIP_DEFLATED) as zipFile:
        for file in os.listdir('merged'):
            zipFile.write('merged/'+file, file)
            print('zipped ' + file + ', buffer size now ' + str(buffer.tell()))            

ValueError: I/O operation on closed file.

In [ ]:
# this writes empty file in COS ...........
with io.BytesIO() as buffer:
    cosObject = cosClient.put_object(Bucket=credentials['BUCKET'], Key='ziptrial.zip', Body=buffer)
    with zipfile.ZipFile(buffer, 'w', compression=zipfile.ZIP_DEFLATED) as zipFile:
        for file in os.listdir('merged'):
            zipFile.write('merged/'+file, file)
            print('zipped ' + file + ', buffer size now ' + str(buffer.tell()))            

In [48]:
# this throws exception ...........
buffer = io.BytesIO()
with zipfile.ZipFile(buffer, 'w', compression=zipfile.ZIP_DEFLATED) as zipFile:
    for file in os.listdir('merged'):
        zipFile.write('merged/'+file, file)
        print('zipped ' + file + ', buffer size now ' + str(buffer.tell()))
            
cosObject = cosClient.put_object(Bucket=credentials['BUCKET'], Key='ziptrial.zip', Body=buffer)
buffer.close()

zipped GeoLite2-City-Locations-en.csv, buffer size now 1263129
zipped GeoLite2-City-Geohashes-en.csv, buffer size now 4475760
zipped ibmData.csv, buffer size now 4979125
zipped GeoLite2-City-Blocks-IPv4.csv, buffer size now 29833259
zipped GeoLite2-City-Blocks-IPv6.csv, buffer size now 40032634


ClientError: An error occurred (BadDigest) when calling the PutObject operation (reached max retries: 4): The Content-MD5 you specified did not match what we received.